In [1]:
from scipy.stats import nbinom, binom, poisson
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os
from scipy.optimize import minimize, Bounds, LinearConstraint, NonlinearConstraint
import inspect

### Relevante Parameter:

array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))

Die Öffungszahlen sind mit einer Poissonverteilung modelliert und die Werte folgender Tabellen haben Einfluss:

1. EM_monat
2. LO_monat

folgende Parameter haben einen Einfluss:
1. alpha_o (1 x anzahl_states array, also state abhängig)
2. beta_o (1 x 1 array, state unabhängig)

In [2]:
# EM_monat aus der HY Zahlen der versendeten Mails ziehen
path = x
filename = 'Pivot_HY_Privat_ohneApple_Langzeitkunde.csv'

df_HY = pd.read_csv(os.path.join(path, filename))

In [3]:
# LO monat
path = x
filename = 'Pivot_LO_Privat_ohneApple_Langzeitkunde.csv'

df_LO = pd.read_csv(os.path.join(path, filename))

### Datenanpassung wie bei der Berechnung der optimalen Parameter

In [4]:
## Alle DataFrame Werte von Float zu int
df_HY = df_HY.astype(int)
df_LO = df_LO.astype(int)

### Sicherstellen dass alle df dieselben Kunden verwenden

In [5]:
# Entfernen der Zeilen aus df_LO, die nicht in df_HY enthalten sind
df_LO = df_LO[df_LO['iid'].isin(df_HY['iid'])]
df_HY = df_HY[df_HY['iid'].isin(df_LO['iid'])]

### Ausreißer bei den Kunden löschen (wie bei den Berechnungen)

In [8]:
kunden_pos = [384, 446, 502, 599, 421, 285, 407, 499, 348, 351, 364, 449, 528, 414]

In [9]:
# Zeilen löschen
df_HY = df_HY.drop(kunden_pos).reset_index(drop = True)
df_LO = df_LO.drop(kunden_pos).reset_index(drop = True)

### Spezifikation des Modells

In [10]:
anzahl_kunden = len(df_HY["iid"])
anzahl_kunden

633

In [11]:
anzahl_monate = len(df_HY.columns[df_HY.columns != 'iid'])
anzahl_monate

15

In [12]:
anzahl_states = 2

### 1. Herauslesen der Parameter aus dem array der gesamten geschätzten Parameter

In [13]:
estimted_params= [ 0.        , -1.84134701,  0.        ,  0.98963076,  0.        ,
        2.68214831,  0.        ,  2.80627766,  0.        , -1.2810976 ,
        0.        ,  0.69838679,  0.        ,  1.45855322,  0.        ,
       -2.13009073,  0.        , -0.04889301,  0.        ,  0.62177964,
        0.33148307,  1.39588074, -1.12147141,  7.70110619,  8.        ,
       -1.20879416,  1.99467476, -1.11065899, -1.89390642,  0.01515651,
        0.17867123, -0.03878318, -0.03539839,  1.03831034]

In [14]:
alpha_size = anzahl_states**2
gamma_size = anzahl_states**2 * 4

alpha = estimted_params[:alpha_size]
gamma = estimted_params[alpha_size:alpha_size+gamma_size]
alpha_o = estimted_params[alpha_size+gamma_size:alpha_size+gamma_size+anzahl_states]
beta_o = estimted_params[alpha_size+gamma_size+anzahl_states:alpha_size+gamma_size+anzahl_states+1]
delta_0 = estimted_params[alpha_size+gamma_size+anzahl_states+1:alpha_size+gamma_size+anzahl_states*2+1]
delta_1 = estimted_params[alpha_size+gamma_size+anzahl_states*2+1:alpha_size+gamma_size+anzahl_states*3+1]
alpha_p = estimted_params[alpha_size+gamma_size+anzahl_states*3+1:alpha_size+gamma_size+anzahl_states*4+1]
beta_p_1 = estimted_params[alpha_size+gamma_size+anzahl_states*4+1:alpha_size+gamma_size+anzahl_states*5+1]
beta_p_2 = estimted_params[alpha_size+gamma_size+anzahl_states*5+1:alpha_size+gamma_size+anzahl_states*6+1]
theta = estimted_params[alpha_size+gamma_size+anzahl_states*6+1:]

In [15]:
alpha_o

[0.33148307, 1.39588074]

In [16]:
beta_o

[-1.12147141]

### Erstellen der erwarteten Öffungen in den jeweiligen Monaten

In [17]:
erwartete_oeffungen = np.zeros((anzahl_kunden, anzahl_states, anzahl_monate))

In [30]:
for kunde in range(0, anzahl_kunden):    
    for state in range(0, anzahl_states):
        for monat in range(1, anzahl_monate +1):
            erwartete_oeffungen[kunde, state, monat-1] =\
                    np.exp(np.log(0.63)*df_HY.iloc[kunde][str(monat)] + alpha_o[state] +\
                           beta_o[0] * np.log(df_LO.iloc[kunde][str(monat)]))

In [31]:
np.sum(erwartete_oeffungen[:,0,:])/erwartete_oeffungen[:,0,:].size

0.4845917285736474

In [32]:
np.sum(erwartete_oeffungen[:,1,:])/erwartete_oeffungen[:,1,:].size

1.4048761236670053